In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

# Fix seed
seed = 65647437836358831880808032086803839626
rng = np.random.default_rng(seed)
rng.random()

0.2607535757716577

#Step 1

Merton Jump Diffusion Model

$$
\begin{equation*}
    dS_t = \left( r - r_j \right) S_t dt + \sigma S_t dZ_t + J_t S_t dN_t
\end{equation*}
$$

Discretized form:

$$
\begin{equation*}
    S_t = S_{t-1} \left( e^{\left(r-r_j-\frac{\sigma^2}{2}\right)dt + \sigma \sqrt{dt} z_t^1}+
    \left(e^{\mu_j+\delta z_t^2}-1 \right) y_t \right)
\end{equation*}
$$

$$
\begin{equation*}
r_j = \lambda \left(e^{\mu_j+\frac{\delta^2}{2}}\right)-1
\end{equation*}
$$

In [ ]:
def merton_jump_diffusion(S0,K,T,r,sigma,mu,delta,lamb):
  Ite = 10000  # Number of simulations (paths)
  M = 100  # Number of steps
  dt = T / M  # Time-step
  SM = np.zeros((M + 1, Ite))
  SM[0] = S0

  # rj
  rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)
  z1 = rng.normal(0, 1,(M + 1, Ite))
  z2 = rng.normal(0, 1,(M + 1, Ite))
  y = rng.poisson(lamb * dt, (M + 1, Ite))

  for t in range(1, M + 1):
      SM[t] = SM[t - 1] * (
          np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
          + (np.exp(mu + delta * z2[t]) - 1) * y[t]
      )
      SM[t] = np.maximum(
          SM[t], 0.00001
      )  # To ensure that the price never goes below zero!

  return SM

In [ ]:
def merton_european_option(SM,K,T,r):

  S_T = SM[-1]

  # Compute payoffs
  call_payoffs = np.maximum(S_T - K, 0)
  put_payoffs = np.maximum(K - S_T, 0)

  # Discount and average
  call_price = np.exp(-r * T) * np.mean(call_payoffs)
  put_price = np.exp(-r * T) * np.mean(put_payoffs)

  return call_price,put_price

λ = 0.75

In [ ]:
mu = -0.5  # intensity mean
delta = 0.22
r = 0.055  # Risk-free rate
sigma = 0.35  # Volatility
T = 3/12  # Maturity/time
S0 = K = 80

In [ ]:
lamb_075 = 0.75  # intensity
SM_lamb_075 = merton_jump_diffusion(S0,K,T,r,sigma,mu,delta,lamb)

In [ ]:
call_075,put_075 = merton_european_option(SM_lamb_075,K,T,r)
print(f"Call price: {call_075:.2f}")
print(f"Put price: {put_075:.2f}")

Call price: 6.81
Put price: 5.81


In [ ]:
assert round(call_075+K*np.exp(-r*T)) == round(S0+put_075)

λ = 0.25

In [ ]:
lamb_025 = 0.25
SM_lamb_025 = merton_jump_diffusion(S0,K,T,r,sigma,mu,delta,lamb)

In [ ]:
call_025,put_025 = merton_european_option(SM_lamb_025,K,T,r)
print(f"Call price: {call_025:.2f}")
print(f"Put price: {put_025:.2f}")

Call price: 6.81
Put price: 5.72


In [ ]:
assert round(call_025+K*np.exp(-r*T)) == round(S0+put_025)

https://people.maths.ox.ac.uk/gilesm/mc/mc/lec7.pdf

$delta = \frac{\partial V}{\partial S}$

In [ ]:
# use finite difference methods
# Delta = [V(S+h) -  V(S-h)] / 2h
def delta_finite_difference(S0,K,T,r,sigma,mu,delta,lamb,increment=0.01):

  delta_S0_up = S0+increment
  delta_S0_down = S0-increment

  SM_delta_up = merton_jump_diffusion(delta_S0_up,K,T,r,sigma,mu,delta,lamb)
  delta_call_up, delta_put_up = merton_european_option(SM_delta_up,K,T,r)

  SM_delta_down = merton_jump_diffusion(delta_S0_down,K,T,r,sigma,mu,delta,lamb)
  delta_call_down, delta_put_down = merton_european_option(SM_delta_down,K,T,r)

  call_delta = (delta_call_up - delta_call_down) / 2*S0*increment
  put_delta = (delta_put_up - delta_put_down) / 2*S0*increment

  return call_delta, put_delta


$gamma = \frac{\partial ^2 V}{\partial S^2}$

In [ ]:
# use finite difference methods
# Gamma = [V(S+h) - 2V(S) + V(S-h)] / h^2
def gamma_finite_difference(S0,K,T,r,sigma,mu,gamma,lamb,increment=0.01):
  delta_S0_up = S0+increment
  delta_S0_down = S0-increment

  # V(S + increment)
  SM_gamma_up = merton_jump_diffusion(delta_S0_up,K,T,r,sigma,mu,delta,lamb)
  gamma_call_up, gamma_put_up = merton_european_option(SM_gamma_up,K,T,r)
  # V(S - increment)
  SM_gamma_down = merton_jump_diffusion(delta_S0_down,K,T,r,sigma,mu,delta,lamb)
  gamma_call_down, gamma_put_down = merton_european_option(SM_gamma_down,K,T,r)

  # V(S)
  SM_gamma_mid = merton_jump_diffusion(S0,K,T,r,sigma,mu,delta,lamb)
  gamma_call_mid, gamma_put_mid = merton_european_option(SM_gamma_mid,K,T,r)

  call_gamma = (gamma_call_up - 2*gamma_call_mid + gamma_call_down) / (S0*increment)**2
  put_gamma = (gamma_put_up - 2*gamma_put_mid + gamma_put_down) / (S0*increment)**2

  return call_gamma, put_gamma


λ = 0.75

In [ ]:
call_delta_075, put_delta_075 = delta_finite_difference(S0,K,T,r,sigma,mu,delta,lamb_075)
print(f"Call delta (λ = 0.75): {call_delta_075:.4f}")
print(f"Put delta (λ = 0.75): {put_delta_075:.4f}")

Call delta (λ = 0.75): 0.0351
Put delta (λ = 0.75): -0.0275


In [ ]:
call_gamma_075, put_gamma_075 = gamma_finite_difference(S0,K,T,r,sigma,mu,delta,lamb_075)
print(f"Call gamma (λ = 0.75): {call_gamma_075:.4f}")
print(f"Put gamma (λ = 0.75): {put_gamma_075:.4f}")

Call gamma (λ = 0.75): 0.1232
Put gamma (λ = 0.75): 0.4446


λ = 0.25

In [ ]:
lamb_025 = 0.25
call_delta_025, put_delta_025 = delta_finite_difference(S0,K,T,r,sigma,mu,delta,lamb_025)
print(f"Call delta (λ = 0.25): {call_delta_025:.4f}")
print(f"Put delta (λ = 0.25): {put_delta_025:.4f}")

Call delta (λ = 0.25): 0.0716
Put delta (λ = 0.25): 0.0248


In [ ]:
call_gamma_025, put_gamma_025 = gamma_finite_difference(S0,K,T,r,sigma,mu,delta,lamb_025)
print(f"Call gamma (λ = 0.25): {call_gamma_025:.4f}")
print(f"Put gamma (λ = 0.25): {put_gamma_025:.4f}")

Call gamma (λ = 0.25): -0.1686
Put gamma (λ = 0.25): -0.1720


American options use LSMC because of early exercise option

https://berthine.github.io/doc/berthine_master-document.pdf , Page 21

https://people.math.ethz.ch/~hjfurrer/teaching/LongstaffSchwartzAmericanOptionsLeastSquareMonteCarlo.pdf , Page 120

In [ ]:
def lsmc_payoff(S_T, K, n_steps, option_type):

    n_paths = S_T.shape[1]
    dt = T / n_steps

    # Initialize cash flow matrix to track when each option is exercised
    cash_flows = np.zeros((n_steps + 1, n_paths))

    # Set terminal payoffs at maturity
    if option_type == 'call':
        cash_flows[n_steps] = np.maximum(S_T[n_steps] - K, 0)
    else:  # put
        cash_flows[n_steps] = np.maximum(K - S_T[n_steps], 0)

    # Track which paths exercise mark with true
    exercised = np.zeros(n_paths, dtype=bool)

    # Backward induction through time steps
    for t in range(n_steps - 1, 0, -1):

        immediate_exercise = np.maximum(S_T[t] - K, 0) if option_type == 'call' else np.maximum(K - S_T[t], 0)

        # Identify in-the-money paths that haven't been exercised yet
        if option_type == 'call':
            itm = (S_T[t] > K) & ~exercised
        else:
            itm = (S_T[t] < K) & ~exercised

        if np.sum(itm) > 0:
            # Get stock prices for in-the-money paths
            X_itm = S_T[t, itm]

            # Calculate discounted future cash flows for regression
            future_cf = np.zeros(n_paths)
            for i in range(t + 1, n_steps + 1):
                future_cf += cash_flows[i] * np.exp(-r * (i - t) * dt)

            # Get future cash flows for in-the-money paths
            Y_itm = future_cf[itm]

            # Fit regression cubic regression
            X_design = np.column_stack([
                np.ones(len(X_itm)),
                X_itm,
                X_itm**2,
                X_itm**3
            ])


            beta = np.linalg.lstsq(X_design, Y_itm, rcond=None)[0]
            continuation_values = X_design @ beta


            # Compare early exercise vs holding
            exercise_decision = immediate_exercise[itm] > continuation_values

            # Get indices of paths to exercise at time t
            exercise_idx = np.where(itm)[0][exercise_decision]

            if len(exercise_idx) > 0:
                # Record cash flow at time t for exercised paths
                cash_flows[t, exercise_idx] = immediate_exercise[exercise_idx]

                # Mark these paths as exercised
                exercised[exercise_idx] = True
                cash_flows[t + 1: n_steps + 1, exercise_idx] = 0

    # Discount all cash flows to time 0
    discounted_payoffs = np.zeros(n_paths)
    for t in range(1, n_steps + 1):
        discounted_payoffs += cash_flows[t] * np.exp(-r * t * dt)

    return discounted_payoffs

In [ ]:
def merton_american_option(SM,K,T,r,option_type):
  Ite = 10000  # Number of simulations (paths)
  M = 100  # Number of steps

  fitted_payoffs= lsmc_payoff(SM,K,M,option_type)
  price = np.mean(fitted_payoffs)

  return price

λ = 0.75

In [ ]:
america_call_merton = merton_american_option(SM_lamb_075,K,T,r,'call')
america_put_merton = merton_american_option(SM_lamb_075,K,T,r,'put')
print(f"American call price: {america_call_merton:.2f}")
print(f"American put price: {america_put_merton:.2f}")

American call price: 7.51
American put price: 6.24


In [ ]:
assert round(america_call_merton+K*np.exp(-r*T)) == round(S0+america_put_merton)

λ = 0.25

In [ ]:
lamb = 0.25  # intensity
america_call_merton = merton_american_option(SM_lamb_025,K,T,r,'call')
america_put_merton = merton_american_option(SM_lamb_025,K,T,r,'put')
print(f"American call price: {america_call_merton:.2f}")
print(f"American put price: {america_put_merton:.2f}")

American call price: 7.02
American put price: 6.17


In [ ]:
assert round(america_call_merton+K*np.exp(-r*T)) == round(S0+america_put_merton)

European Down-and-In (Knock In Barrier) Option

In [ ]:
def down_in_barrier(S0,K,T,r,sigma,mu,delta,lamb, barrier):

    S = merton_jump_diffusion(S0,K,T,r,sigma,mu,delta,lamb)
    S_T = S[-1]

    barrier_hit = np.any(S <= barrier, axis=0)

    # Calculate payoff (only valid if barrier is hit)
    call_payoff = np.where(barrier_hit, np.maximum(S_T - K, 0), 0)
    put_payoff =  np.where(barrier_hit, np.maximum(K - S_T, 0), 0)

    # Calculate option price
    call_price = np.exp(-r * T) * np.mean(call_payoff)
    put_price = np.exp(-r * T) * np.mean(put_payoff)

    return call_price, put_price

In [ ]:
mu = -0.5  # intensity mean
delta = 0.22
r = 0.055  # Risk-free rate
sigma = 0.35  # Volatility
T = 3/12  # Maturity/time
S0 = 80
K = barrier = 65

In [ ]:
dai_call, dai_put = down_in_barrier(S0,K,T,r,sigma,mu,delta,lamb,barrier)
print(f"European DAI Barrier call price: {dai_call:.2f}")
print(f"European DAI Barrier put price: {dai_put:.2f}")

European DAI Barrier call price: 0.53
European DAI Barrier put price: 1.43
